In [2]:
a = [1,2,3,4]

match a:
    case x,*xs:
        print(x)
        print(xs)

1
[2, 3, 4]


In [43]:
from typing import Callable, Generator

def foldl(f:Callable[[int,int],int],x0:int,v:list[int])->Generator[int,None,None]:
    match v:
        case []: yield x0
        case x,*xs: yield from foldl(f,f(x0,x),xs)

In [54]:
a=[x for x in range(1,5)]
foo=foldl((lambda x,y: x+y),0,a)
for i in foo:
    print(i)


10


In [103]:
def echo_round() -> Generator[int, float, str]:
    sent = yield 0
    while sent >= 0:
        sent = yield round(sent)
    return 'Done'

In [120]:
foo=echo_round()

Not sure about the following

In [122]:
e = None
for _ in range(4):
    v=next(foo,e)
    if v!=None: print(v)
    else: break

In [4]:
[y for y in range(20) if y<10]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [41]:
from typing import TypeVar

A = TypeVar('A',bound=int|float|str)

def qsort(v:list[A])->list[A]:
    match v:
        case []: return v
        case x,*xs:
            small=[y for y in xs if y<x]
            mid = [y for y in xs if y==x]+[x]
            large = [y for y in xs if y>x]
            return qsort(small)+mid+qsort(large)
        case _: return v # to make the type checker happy

In [42]:
qsort([1,3,2,7,6,4,5])

[1, 2, 3, 4, 5, 6, 7]

In [60]:
from functools import singledispatch
@singledispatch
def mempty(a):
    raise NotImplementedError("Not implemented for" + a)

@singledispatch
def mappend(a, b):
    raise NotImplementedError("Not implemented for" + a)

In [65]:
@mempty.register(list)
def _(a):
    return []

@mappend.register(list)
def _(a,b):
    return a + b

@mempty.register(tuple)
def _(a):
    return ()

@mappend.register(tuple)
def _(a,b):
    return a+b

In [62]:
mappend([1,2,3],[4,5,6])

[1, 2, 3, 4, 5, 6]

In [66]:
mappend((1,2,3),(4,5,6))

(1, 2, 3, 4, 5, 6)

In [70]:
@mempty.register(None.__class__)
def _(a):
    return None

@mappend.register(None.__class__)
def _(a,b):
    return None

In [73]:
mappend(None,None)==None

True

In [75]:
def mconcat(l):
    acc = l[0]
    for x in l[1:]:
        acc = mappend(acc,x)
    return acc

@mappend.register(mconcat.__class__)
def _(a,b):
  def result(*x):
    a_r= a(*x)
    b_r=b(*x)
    return mappend(a_r,b_r)
  return result

In [76]:
mconcat([[1,2],[3,4,5]])

[1, 2, 3, 4, 5]

In [77]:
def phello(arg):
  print("Hello",arg)
  
def phi(arg):
  print("Hi",arg)

mappend(phello,phi)("World!")

Hello World!
Hi World!


In [79]:
mappend(phello,phi)("world")

Hello world
Hi world


In [82]:
import operator

operator.add([1,2],[3,4])

[1, 2, 3, 4]